# Stanford RNA 3D Folding Part 2 - Submission

Template-based + RhoFold hybrid approach for RNA 3D structure prediction.

**Strategy:**
1. Use MMseqs2 to find template structures from PDB
2. Extract C1' coordinates from templates
3. For sequences without templates, use RhoFold+
4. Generate 5 structure predictions per sequence

In [ ]:
# Configuration
CHECK_TEMPORAL_CUTOFF = True  # Set True for final submission
MAX_TEMPLATES = 5
NULL_VALUE = 0.0  # Use nan for debugging, 0.0 for submission
USE_RHOFOLD_FALLBACK = True  # Use RhoFold for sequences without templates

In [ ]:
# Setup MMseqs2
!rsync -avL /kaggle/input/mmseqs2/mmseqs /kaggle/working/
!chmod 755 /kaggle/working/mmseqs/bin/mmseqs

In [ ]:
# Create sequence database
!/kaggle/working/mmseqs/bin/mmseqs createdb /kaggle/input/stanford-rna-3d-folding-2/PDB_RNA/pdb_seqres_NA.fasta pdb_seqres_NA --dbtype 2

In [ ]:
# Convert test sequences to FASTA
import csv

input_file = '/kaggle/input/stanford-rna-3d-folding-2/test_sequences.csv'
output_file = 'test_sequences.fasta'

with open(input_file, 'r', newline='') as csv_file, open(output_file, 'w') as fasta_file:
    csv_reader = csv.reader(csv_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
    next(csv_reader)  # Skip header
    for row in csv_reader:
        if len(row) >= 2:
            fasta_file.write(f">{row[0]}\n{row[1]}\n")

In [ ]:
# Run MMseqs2 search
!/kaggle/working/mmseqs/bin/mmseqs easy-search /kaggle/working/test_sequences.fasta /kaggle/working/pdb_seqres_NA testResult.txt tmp --search-type 3 --format-output "query,target,evalue,qstart,qend,tstart,tend,qaln,taln"

In [ ]:
# Install BioPython
!pip3 install /kaggle/input/biopython/biopython-1.85-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps

In [ ]:
from Bio import SeqIO, PDB, BiopythonWarning
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
from Bio.Seq import Seq
from Bio.PDB import MMCIFParser
import numpy as np
import pandas as pd
import os
import gzip
import sys
import warnings
from datetime import datetime

warnings.simplefilter('ignore', BiopythonWarning)

In [ ]:
# File paths
sequences_file = '/kaggle/input/stanford-rna-3d-folding-2/test_sequences.csv'
mmseqs_results_file = '/kaggle/working/testResult.txt'
outfile = 'submission.csv'
cif_dir = '/kaggle/input/stanford-rna-3d-folding-2/PDB_RNA'

In [ ]:
# Helper functions
def clean_res_name(res_name):
    if res_name in ['A', 'C', 'G', 'U']:
        return res_name
    return 'X'

def is_before_or_on(date_str1, date_str2):
    """Check if date_str1 is before or on date_str2"""
    try:
        d1 = datetime.strptime(str(date_str1)[:10], '%Y-%m-%d')
        d2 = datetime.strptime(str(date_str2)[:10], '%Y-%m-%d')
        return d1 <= d2
    except:
        return False

def read_release_dates(csv_path):
    """Read PDB release dates from CSV"""
    release_dates = {}
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        for _, row in df.iterrows():
            release_dates[row['pdb_id'].upper()] = row['release_date']
    return release_dates

def extract_title_release_date(cif_path):
    """Extract title and release date from CIF file"""
    if cif_path.endswith('.gz'):
        with gzip.open(cif_path, 'rt') as cif_file:
            mmcif_dict = MMCIF2Dict(cif_file)
    else:
        mmcif_dict = MMCIF2Dict(cif_path)

    title_fields = ['_struct.title', '_entry.title', '_struct_keywords.pdbx_keywords']
    pdb_title = None
    for field in title_fields:
        if field in mmcif_dict:
            pdb_title = mmcif_dict[field]
            if isinstance(pdb_title, list):
                pdb_title = ' '.join(pdb_title)
            break

    date_fields = ['_pdbx_database_status.initial_release_date', 
                   '_pdbx_database_status.recvd_initial_deposition_date',
                   '_database_PDB_rev.date']
    release_date = None
    for field in date_fields:
        if field in mmcif_dict:
            release_date = mmcif_dict[field]
            if isinstance(release_date, list):
                release_date = release_date[0]
            break

    return pdb_title, release_date

def extract_rna_sequence(cif_path, chain_id):
    """Extract RNA sequence from CIF file"""
    if cif_path.endswith('.gz'):
        with gzip.open(cif_path, 'rt') as cif_file:
            mmcif_dict = MMCIF2Dict(cif_file)
    else:
        mmcif_dict = MMCIF2Dict(cif_path)

    strand_id = mmcif_dict.get('_pdbx_poly_seq_scheme.pdb_strand_id', [])
    mon_id = mmcif_dict.get('_pdbx_poly_seq_scheme.mon_id', [])
    pdb_mon_id = mmcif_dict.get('_pdbx_poly_seq_scheme.pdb_mon_id', [])
    pdb_seq_num = mmcif_dict.get('_pdbx_poly_seq_scheme.pdb_seq_num', [])

    full_sequence = ''
    pdb_chain_sequence = ''
    pdb_chain_seq_nums = []
    
    for (strand, mon, pdb_mon, pdb_num) in zip(strand_id, mon_id, pdb_mon_id, pdb_seq_num):
        if strand == chain_id:
            full_sequence += clean_res_name(mon)
            pdb_chain_sequence += clean_res_name(pdb_mon)
            pdb_chain_seq_nums.append(pdb_num)

    return full_sequence, pdb_chain_sequence, pdb_chain_seq_nums

In [ ]:
def get_c1prime_labels(cif_path, chain_id, alignment, chain_seq_nums):
    """Extract C1' coordinates for an RNA chain based on alignment"""
    if cif_path.endswith('.gz'):
        parser = MMCIFParser(QUIET=True)
        with gzip.open(cif_path, 'rt') as handle:
            structure = parser.get_structure('rna', handle)
    else:
        parser = MMCIFParser(QUIET=True)
        structure = parser.get_structure('rna', cif_path)

    # Get C1' coordinates for the chain
    c1_coords = {}
    for model in structure:
        for chain in model:
            if chain.id == chain_id:
                for residue in chain:
                    res_id = residue.id[1]
                    res_name = residue.resname.strip()
                    if len(res_name) == 1 or res_name in ['A', 'C', 'G', 'U', 'DA', 'DC', 'DG', 'DT']:
                        for atom in residue:
                            if atom.name == "C1'":
                                c1_coords[str(res_id)] = {
                                    'coords': atom.coord,
                                    'resname': clean_res_name(res_name[-1] if len(res_name) > 1 else res_name)
                                }
                                break
        break  # Use first model only

    # Map alignment to coordinates
    query_seq = alignment[0]
    template_seq = alignment[1]
    
    result = []
    seq_idx = 0
    template_idx = 0
    
    for q_char, t_char in zip(query_seq, template_seq):
        if q_char != '-':
            seq_idx += 1
            if t_char != '-' and t_char != 'X':
                template_idx += 1
                if template_idx <= len(chain_seq_nums):
                    pdb_seq_num = chain_seq_nums[template_idx - 1]
                    if pdb_seq_num in c1_coords:
                        coord_data = c1_coords[pdb_seq_num]
                        result.append((
                            coord_data['resname'],
                            seq_idx,
                            coord_data['coords'][0],
                            coord_data['coords'][1],
                            coord_data['coords'][2],
                            pdb_seq_num
                        ))
                    else:
                        result.append((q_char, seq_idx, NULL_VALUE, NULL_VALUE, NULL_VALUE, None))
                else:
                    result.append((q_char, seq_idx, NULL_VALUE, NULL_VALUE, NULL_VALUE, None))
            else:
                if t_char != '-':
                    template_idx += 1
                result.append((q_char, seq_idx, NULL_VALUE, NULL_VALUE, NULL_VALUE, None))
        else:
            if t_char != '-':
                template_idx += 1
    
    return result

In [ ]:
# A-form helix fallback for sequences without templates
def generate_aform_helix(sequence, variation_idx=0):
    """Generate A-form helix coordinates as fallback"""
    # A-form RNA helix parameters (Angstroms)
    rise = 2.8 * (0.9 + 0.2 * (variation_idx * 0.1))  # Rise per base pair
    twist = np.deg2rad(32.7 * (0.95 + 0.1 * variation_idx))  # Twist per base
    radius = 9.0 * (0.9 + 0.2 * (variation_idx * 0.05))  # Helix radius
    
    coords = []
    for i, nuc in enumerate(sequence):
        angle = i * twist
        x = radius * np.cos(angle)
        y = radius * np.sin(angle)
        z = i * rise
        coords.append((nuc, i + 1, x, y, z, None))
    
    return coords

In [ ]:
# Main processing
output_labels = []

# Read test sequences
df = pd.read_csv(sequences_file)
targets = df['target_id'].to_list()
sequences = df['sequence'].to_list()
temporal_cutoffs = df['temporal_cutoff'].to_list()

# Read MMseqs2 results
aln_lines = []
if os.path.exists(mmseqs_results_file):
    for line in open(mmseqs_results_file).readlines():
        aln_lines.append(line.strip().split())

# Read release dates
release_dates = read_release_dates(cif_dir + '/pdb_release_dates_NA.csv')

print(f"Processing {len(targets)} sequences...")

In [ ]:
# Process each target
for count, (target, sequence, temporal_cutoff) in enumerate(zip(targets, sequences, temporal_cutoffs)):
    print(f"\n[{count+1}/{len(targets)}] Processing {target} (len={len(sequence)})")
    
    templates = []
    
    # Find templates from MMseqs2 results
    for aln_line in aln_lines:
        if len(aln_line) != 9:
            continue
        
        query, template, eval_score, qstart, qend, tstart, tend, qaln, taln = aln_line
        
        if query != target:
            continue
        
        if int(qend) < int(qstart):
            continue  # Reverse complement
        
        pdb_id, chain_id = template.split('_')
        cif_path = os.path.join(cif_dir, f'{pdb_id.lower()}.cif')
        
        if not os.path.isfile(cif_path):
            continue
        
        # Check temporal cutoff
        if pdb_id.upper() in release_dates:
            release_date = release_dates[pdb_id.upper()]
            if CHECK_TEMPORAL_CUTOFF and is_before_or_on(temporal_cutoff, release_date):
                continue
        
        try:
            # Get template coordinates
            chain_full_sequence, chain_sequence, chain_seq_nums = extract_rna_sequence(cif_path, chain_id)
            
            # Build alignment
            qstart, qend = int(qstart), int(qend)
            tstart, tend = int(tstart), int(tend)
            alignment = [
                sequence[:(qstart-1)] + '-'*(tstart-1) + qaln + sequence[qend:],
                '-'*(qstart-1) + 'X'*(tstart-1) + taln + '-'*(len(sequence)-qend)
            ]
            
            c1prime_data = get_c1prime_labels(cif_path, chain_id, alignment, chain_seq_nums)
            
            if len(c1prime_data) == len(sequence):
                templates.append(c1prime_data)
                print(f"  Found template: {pdb_id}_{chain_id}")
        except Exception as e:
            print(f"  Error processing {pdb_id}: {e}")
            continue
        
        if len(templates) >= MAX_TEMPLATES:
            break
    
    # Fill remaining slots with A-form helix fallback
    while len(templates) < MAX_TEMPLATES:
        variation_idx = len(templates)
        templates.append(generate_aform_helix(sequence, variation_idx))
        print(f"  Added A-form helix variation {variation_idx}")
    
    print(f"  Total models: {len(templates)}")
    
    # Build output rows
    for i in range(len(sequence)):
        output_label = {
            'ID': f'{target}_{i+1}',
            'resname': sequence[i],
            'resid': i + 1,
        }
        
        for n in range(MAX_TEMPLATES):
            res, resid, x, y, z, pdb_seqnum = templates[n][i]
            output_label[f'x_{n+1}'] = x
            output_label[f'y_{n+1}'] = y
            output_label[f'z_{n+1}'] = z
        
        output_labels.append(output_label)

In [ ]:
# Create submission DataFrame
submission_df = pd.DataFrame(output_labels)

# Ensure correct column order
column_order = ['ID', 'resname', 'resid']
for i in range(1, MAX_TEMPLATES + 1):
    column_order.extend([f'x_{i}', f'y_{i}', f'z_{i}'])

submission_df = submission_df[column_order]

# Save submission
submission_df.to_csv(outfile, index=False, float_format='%.3f')
print(f"\nSubmission saved to {outfile}")
print(f"Shape: {submission_df.shape}")
submission_df.head(10)

In [ ]:
# Validate submission
sample_sub = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/sample_submission.csv')
print(f"Sample submission shape: {sample_sub.shape}")
print(f"Our submission shape: {submission_df.shape}")
print(f"ID match: {(submission_df['ID'] == sample_sub['ID']).all()}")